# Lab3 - Assignment Sentiment

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook describes the LAB-3 assignment of the Text Mining course. It is about sentiment analysis.

The aims of the assignment are:
* Learn how to run a rule-based sentiment analysis module (VADER)
* Learn how to run a machine learning sentiment analysis module (Scikit-Learn/ Naive Bayes)
* Learn how to run scikit-learn metrics for the quantitative evaluation
* Learn how to perform and interpret a quantitative evaluation of the outcomes of the tools (in terms of Precision, Recall, and F<sub>1</sub>)
* Learn how to evaluate the results qualitatively (by examining the data) 
* Get insight into differences between the two applied methods
* Get insight into the effects of using linguistic preprocessing
* Be able to describe differences between the two methods in terms of their results
* Get insight into issues when applying these methods across different  domains

In this assignment, you are going to create your own gold standard set from 50 tweets. You will the VADER and scikit-learn classifiers to these tweets and evaluate the results by using evaluation metrics and inspecting the data.

We recommend you go through the notebooks in the following order:
* **Read the assignment (see below)**
* **Lab3.2-Sentiment-analysis-with-VADER.ipynb**
* **Lab3.3-Sentiment-analysis.with-scikit-learn.ipynb**
* **Answer the questions of the assignment (see below) using the provided notebooks and submit**

In this assignment you are asked to perform both quantitative evaluations and error analyses:
* a quantitative evaluation concerns the scores (Precision, Recall, and F<sub>1</sub>) provided by scikit's classification_report. It includes the scores per category, as well as micro and macro averages. Discuss whether the scores are balanced or not between the different categories (positive, negative, neutral) and between precision and recall. Discuss the shortcomings (if any) of the classifier based on these scores
* an error analysis regarding the misclassifications of the classifier. It involves going through the texts and trying to understand what has gone wrong. It servers to get insight in what could be done to improve the performance of the classifier. Do you observe patterns in misclassifications?  Discuss why these errors are made and propose ways to solve them.

## Credits
The notebooks in this block have been originally created by [Marten Postma](https://martenpostma.github.io) and [Isa Maks](https://research.vu.nl/en/persons/e-maks). Adaptations were made by [Filip Ilievski](http://ilievski.nl).

## Part I: VADER assignments


### Preparation (nothing to submit):
To be able to answer the VADER questions you need to know how the tool works. 
* Read more about the VADER tool in [this blog](http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html).  
* VADER provides 4 scores (positive, negative, neutral, compound). Be sure to understand what they mean and how they are calculated.
* VADER uses rules to handle linguistic phenomena such as negation and intensification. Be sure to understand which rules are used, how they work, and why they are important.
* VADER makes use of a sentiment lexicon. Have a look at the lexicon. Be sure to understand which information can be found there (lemma?, wordform?, part-of-speech?, polarity value?, word meaning?) What do all scores mean? https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vader_lexicon.txt) 


### [3.5 points] Question1:

Regard the following sentences and their output as given by VADER. Regard sentences 1 to 7, and explain the outcome **for each sentence**. Take into account both the rules applied by VADER and the lexicon that is used. You will find that some of the results are reasonable, but others are not. Explain what is going wrong or not when correct and incorrect results are produced. 

```
INPUT SENTENCE 1 I love apples
VADER OUTPUT {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

INPUT SENTENCE 2 I don't love apples
VADER OUTPUT {'neg': 0.627, 'neu': 0.373, 'pos': 0.0, 'compound': -0.5216}

INPUT SENTENCE 3 I love apples :-)
VADER OUTPUT {'neg': 0.0, 'neu': 0.133, 'pos': 0.867, 'compound': 0.7579}

INPUT SENTENCE 4 These houses are ruins
VADER OUTPUT {'neg': 0.492, 'neu': 0.508, 'pos': 0.0, 'compound': -0.4404}

INPUT SENTENCE 5 These houses are certainly not considered ruins
VADER OUTPUT {'neg': 0.0, 'neu': 0.51, 'pos': 0.49, 'compound': 0.5867}

INPUT SENTENCE 6 He lies in the chair in the garden
VADER OUTPUT {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.4215}

INPUT SENTENCE 7 This house is like any house
VADER OUTPUT {'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.3612}
```

Sentence 1: 'love' only has positive valence, 'I' and 'apples' are neutral words and the negative score is 0 because there are no negative words. The result is correct.

Sentence 2: 'love' has a positive valence, but it sees the negative "don't" and flips the love segment into a negative.  'I' and 'apples' are neutral words. The result is correct.

Sentence 3: VADER recognises the emoticon ':-)' as a positive valence, alongside 'love'. Similarly to the first sentence, it has a high positive score, but the smiley face gives it a stronger positive valence. The result is correct.

Sentence 4: 'ruins' is regarded as a negative word, synonymous with words like 'destroyed', it regards the sentence as negative overall, while the rest are neutral words, hence the high neutral score. The result is correct.

Sentence 5: 'ruins' is a negative word, 'not' flips the statement and 'certainly' amplifies the valence. The rest are neutral words. The result is correct.

Sentence 6: 'lies' is considered negative, the rest are neutral, hence the big neutral score. The result is incorrect, because VADER did not understand the context of 
the sentence.

Sentence 7: 'like' is regarded as a positive word and increases the overall score, however the result should be neutral, due to the context being a neutral sentence. The compound should be 0.

### [Points: 2.5] Exercise 2: Collecting 50 tweets for evaluation
Collect 50 tweets. Try to find tweets that are interesting for sentiment analysis, e.g., very positive, neutral, and negative tweets. These could be your own tweets (typed in) or collected from the Twitter stream. If you have trouble accessing Twitter, try to find an existing dataset (on websites like kaggle or huggingface).

We will store the tweets in the file **my_tweets.json** (use a text editor to edit).
For each tweet, you should insert:
* sentiment analysis label: negative | neutral | positive (this you determine yourself, this is not done by a computer)
* the text of the tweet
* the Tweet-URL

from:
```
    "1": {
        "sentiment_label": "",
        "text_of_tweet": "",
        "tweet_url": "",
```
to:
```
"1": {
        "sentiment_label": "positive",
        "text_of_tweet": "All across America people chose to get involved, get engaged and stand up. Each of us can make a difference, and all of us ought to try. So go keep changing the world in 2018.",
        "tweet_url" : "https://twitter.com/BarackObama/status/946775615893655552",
    },
```

You can load your tweets with human annotation in the following way.

In [110]:
import json

In [111]:
my_tweets = json.load(open('my_tweets.json'))

In [112]:
for id_, tweet_info in my_tweets.items():
    print(id_, tweet_info)
    break

1 {'sentiment_label': 'Neutral', 'text_of_tweet': '_ricardo road movies indies rule  haha', 'tweet_url': ''}


### [5 points] Question 3:

Run VADER on your own tweets (see function **run_vader** from notebook **Lab2-Sentiment-analysis-using-VADER.ipynb**). You can use the code snippet below this explanation as a starting point. 
* [2.5 points] a. Perform a quantitative evaluation. Explain the different scores, and explain which scores are most relevant and why.
* [2.5 points] b. Perform an error analysis: select 10 positive, 10 negative and 10 neutral tweets that are not correctly classified and try to understand why. Refer to the VADER-rules and the VADER-lexicon. Of course, if there are less than 10 errors for a category, you only have to check those. For example, if there are only 5 errors for positive tweets, you just describe those.

In [113]:
def vader_output_to_label(vader_output):
    """
    map vader output e.g.,
    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
    to one of the following values:
    a) positive float -> 'positive'
    b) 0.0 -> 'neutral'
    c) negative float -> 'negative'
    
    :param dict vader_output: output dict from vader
    
    :rtype: str
    :return: 'negative' | 'neutral' | 'positive'
    """
    compound = vader_output['compound']
    
    if compound < 0:
        return 'negative'
    elif compound == 0.0:
        return 'neutral'
    elif compound > 0.0:
        return 'positive'
    
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.0}) == 'neutral'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.01}) == 'positive'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': -0.01}) == 'negative'

In [114]:
import numpy
import nltk
from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import spacy
import pathlib
import sklearn
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from collections import Counter

nlp = spacy.load('en_core_web_sm')
vader_model = SentimentIntensityAnalyzer()

def run_vader(textual_unit, 
              lemmatize=False, 
              parts_of_speech_to_consider=None,
              verbose=1):
    """
    Run VADER on a sentence from spacy
    
    :param str textual unit: a textual unit, e.g., sentence, sentences (one string)
    (by looping over doc.sents)
    :param bool lemmatize: If True, provide lemmas to VADER instead of words
    :param set parts_of_speech_to_consider:
    -None or empty set: all parts of speech are provided
    -non-empty set: only these parts of speech are considered.
    :param int verbose: if set to 1, information is printed
    about input and output
    
    :rtype: dict
    :return: vader output dict
    """
    doc = nlp(textual_unit)
        
    input_to_vader = []

    for sent in doc.sents:
        for token in sent:

            to_add = token.text

            if lemmatize:
                to_add = token.lemma_

                if to_add == '-PRON-': 
                    to_add = token.text

            if parts_of_speech_to_consider:
                if token.pos_ in parts_of_speech_to_consider:
                    input_to_vader.append(to_add) 
            else:
                input_to_vader.append(to_add)

    scores = vader_model.polarity_scores(' '.join(input_to_vader))
    
    if verbose >= 1:
        print()
        print('INPUT SENTENCE', sent)
        print('INPUT TO VADER', input_to_vader)
        print('VADER OUTPUT', scores)

    return scores

In [115]:
from sklearn.metrics import classification_report

tweets = []
all_vader_output = []
gold = []

# settings (to change for different experiments)
to_lemmatize = True 
pos = set()

for id_, tweet_info in my_tweets.items():
    the_tweet = tweet_info['text_of_tweet']
    vader_output = run_vader(the_tweet, to_lemmatize,verbose= 1)# run vader
    vader_label = vader_output_to_label(vader_output)# convert vader output to category
    
    tweets.append(the_tweet)
    all_vader_output.append(vader_label)
    gold.append(tweet_info['sentiment_label'])
    
# use scikit-learn's classification report
# Convert all gold labels to lowercase
gold = [label.lower() for label in gold]

all_vader_output = [label.lower() for label in all_vader_output]

report = classification_report(gold, all_vader_output)

print("Classification Report (VADER Predictions vs. Gold Labels):")
print(report)



INPUT SENTENCE _ricardo road movies indies rule  haha
INPUT TO VADER ['_', 'ricardo', 'road', 'movie', 'indie', 'rule', ' ', 'haha']
VADER OUTPUT {'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'compound': 0.4588}

INPUT SENTENCE just FYI.
INPUT TO VADER [' ', 'p.s', '.', ':', 'uv', 'ray', 'be', 'just', 'as', 'strong', 'with', 'cloud', 'as', 'with', 'regular', 'sun', ',', 'sometimes', 'strong', '.', 'just', 'FYI', '.']
VADER OUTPUT {'neg': 0.0, 'neu': 0.605, 'pos': 0.395, 'compound': 0.8455}

INPUT SENTENCE _jackson already there`s none left  back to cabbage soup
INPUT TO VADER ['_', 'jackson', 'already', 'there`s', 'none', 'leave', ' ', 'back', 'to', 'cabbage', 'soup']
VADER OUTPUT {'neg': 0.0, 'neu': 0.875, 'pos': 0.125, 'compound': 0.0382}

INPUT SENTENCE  The rest of the week is less than 2.5 hours long
INPUT TO VADER [' ', 'the', 'rest', 'of', 'the', 'week', 'be', 'less', 'than', '2.5', 'hour', 'long']
VADER OUTPUT {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

INPUT SENTENCE I 

## Question 3.a
Well the results for neutral tweets are low, that could be due neutral being ambiguous for VADER. However, the F1 score is a good way of measuring the overall performance of VADER. VADER performed best on its negative tweets, as seen by the 0.67 F1 score, however it made a lot of mistakes, as the average f1 score is close to 0.5.

## Question 3.b

### False-Negative for Neutral Tweets
- 1. _ricardo road movies indies rule haha
    - Predicted as: Positive.
    - “haha” is a positive indicator in VADER’s lexicon. 
- 2. p.s.: UV rays are just as strong with clouds... just FYI.
    - Positive
    - "strong" has a positive valence for VADER

- 3. _jackson already there’s none left back to cabbage soup
    - Positive
    - Maybe due to the already? or there was a mix of positive and negative words.

- 4. Damnation. I just woke up.
    - Negative
    - "Damnation" has strong negative valence
 
- 5. ...so lucky!...i think we're dangerous(?
    - Positive
    - "lucky" was the main cause, which overshadowed "dangerous"
 
- 6. Friday!!!!!!!!!! Wooo and nothing to do... Nuggets better pull it off
    - Positive
    - In context it makes sense, but VADER picks up "Friday", "!" and "Woo" with positive valence

- 7. Home from Whitney`s. No sleep. Church in a bit. Dangggg....I`m tired. But it was worth it"
    - Negative
    - "Dang" "No" and "tired" probably caused the negative result

- 8. _uk Yeah, but it’s Myst!
    - Positive
    - "Yeah" is a positive valence word.

- 9. Munch away, my dear Watson, Munch away
    - Positive
    - Possibly due to "dear"?

- 10. My name is toggling in and out of Google Suggestions Help me by searching more for 'Ganesh Jaju' and clicking on some link
    - Positive
    - Unknown why it would treat this as positive.


## False-Negative for Negative Tweets

- 1. mondayyyyyy
  - Neutral
  - VADER sees “mondayyyyyy” as a neutral word

- 2. I really want to like Morningwood, but they provided the theme song for &quot;Daisy of Love&quot; which is almost unforgivable in my book!
  - Positive
  - "want to like", "really", "Love" are all positive valence words.

- 3. FRANNNNNNNNNY how was your birthday?? You look gorg in the photos, wish I could have been there to party with you  love u xx
  - Positive
  - Actually this one is a mistake of where i recieved the tweets from. So VADER correctly predicted positive here.

- 4. I got yelled at for rapping
  - Neutral
  - "yelled at" maybe did not produce much of a negative valence

- 5. the climbing wall at one of them. Good article. Sperrins aren't high on the NITB agenda so we only get a few FAM trips 
  - Positive
  - "good article" could lead to positive result

- 6. also wish I could be at #biorama2 but found out... too late
  - Positive
  - "wish" could be a strong positive valence

- 7. it's so hot
  - Neutral
  - "hot" is not necessarily negative for VADER


### False-Negatives for Positive Tweets

- 1. Named my new kitty Zeus
  - Neutral
  - "kitty" is treated as neutral, "new" also.

- 2. sims 3 isnt that great. i still lurve my good ol' modded versions of sims 2.
  - Negative
  - "isn't great" definitely, and also the incorrect grammar on "lurve" meaning VANDER possibly did not recognise the word love

- 3. Did you know he was born in San Antonio?!
  - Neutral
  - The "?!" made it neutral.

- 4. Last day of school - enough said
  - Neutral
  - VANDER does not understand context, and no positive words were used.

- 5. What an eventful day
  - Neutral
  - "eventful" recognised as a neutral word




### [4 points] Question 4:
Run VADER on the set of airline tweets with the following settings:

* Run VADER (as it is) on the set of airline tweets 
* Run VADER on the set of airline tweets after having lemmatized the text
* Run VADER on the set of airline tweets with only adjectives
* Run VADER on the set of airline tweets with only adjectives and after having lemmatized the text
* Run VADER on the set of airline tweets with only nouns
* Run VADER on the set of airline tweets with only nouns and after having lemmatized the text
* Run VADER on the set of airline tweets with only verbs
* Run VADER on the set of airline tweets with only verbs and after having lemmatized the text

* [1 point] a. Generate for all separate experiments the classification report, i.e., Precision, Recall, and F<sub>1</sub> scores per category as well as micro and macro averages. **Use a different code cell (or multiple code cells) for each experiment.**
* [3 points] b. Compare the scores and explain what they tell you.
* - Does lemmatisation help? Explain why or why not.
* - Are all parts of speech equally important for sentiment analysis? Explain why or why not.

In [116]:
# Your code here
# Preliminaries
cwd = pathlib.Path.cwd()
airline_tweets_folder = cwd.joinpath('airlinetweets')
path = str(airline_tweets_folder)

airline_tweets = load_files(path)


In [117]:
import os

# List to store all tweets with their sentiment
all_airline_tweets = []

# Iterate through each folder inside airlinetweets
for folder_name in os.listdir(airline_tweets_folder):
    folder_path = airline_tweets_folder / folder_name
    if folder_path.is_dir():
        # Iterate through each file in the current folder
        for file_name in os.listdir(folder_path):
            file_path = folder_path / file_name
            if file_path.is_file():
                # Read the text from the file and add it to the list
                with open(file_path, 'r', encoding='utf-8') as file:
                    tweet_text = file.read().strip()
                    all_airline_tweets.append([folder_name, tweet_text])


In [118]:
def run_vader_on_airline_tweets(lemmatize=False, pos=None):
    gold_airline = []
    all_vader_output_airline = []
    tweets_airline = []

    for tweet in all_airline_tweets:
        tweet_text = tweet[1]
        vader_output_airline = run_vader(tweet_text, lemmatize, pos, verbose=0)
        vader_label_airline = vader_output_to_label(vader_output_airline)

        tweets_airline.append(tweet_text)
        all_vader_output_airline.append(vader_label_airline)
        gold_airline.append(tweet[0])

    gold_airline = [label.lower() for label in gold_airline]
    all_vader_output_airline = [label.lower() for label in all_vader_output_airline]

    report_airline = classification_report(gold_airline, all_vader_output_airline)
    return report_airline


In [119]:
# As-Is
print("Airline Report : As-Is")
report_airline = run_vader_on_airline_tweets(lemmatize=False, pos=set())
print(report_airline)


Airline Report : As-Is
              precision    recall  f1-score   support

    negative       0.80      0.51      0.63      1750
     neutral       0.60      0.51      0.55      1515
    positive       0.56      0.88      0.68      1490

    accuracy                           0.63      4755
   macro avg       0.65      0.64      0.62      4755
weighted avg       0.66      0.63      0.62      4755



In [120]:
# Lemmatized
print("Airline Report : Lemmatized")
report_airline_lemmatized = run_vader_on_airline_tweets(lemmatize=True, pos=set())
print(report_airline_lemmatized)


Airline Report : Lemmatized
              precision    recall  f1-score   support

    negative       0.79      0.52      0.63      1750
     neutral       0.60      0.49      0.54      1515
    positive       0.56      0.88      0.68      1490

    accuracy                           0.62      4755
   macro avg       0.65      0.63      0.62      4755
weighted avg       0.65      0.62      0.62      4755



In [121]:
# As-Is with only Adjectives
print("Airline Report : As-Is with ADJ POS")
report_airline_adj = run_vader_on_airline_tweets(lemmatize=False, pos={'ADJ'})
print(report_airline_adj)

Airline Report : As-Is with ADJ POS
              precision    recall  f1-score   support

    negative       0.86      0.20      0.33      1750
     neutral       0.40      0.89      0.55      1515
    positive       0.67      0.44      0.53      1490

    accuracy                           0.50      4755
   macro avg       0.64      0.51      0.47      4755
weighted avg       0.65      0.50      0.46      4755



In [122]:
# Lemmatized with only Adjectives
print("Airline Report : Lemmatized with ADJ POS")
report_airline_lemmatized_adj = run_vader_on_airline_tweets(lemmatize=True, pos={'ADJ'})
print(report_airline_lemmatized_adj)

Airline Report : Lemmatized with ADJ POS
              precision    recall  f1-score   support

    negative       0.86      0.20      0.33      1750
     neutral       0.40      0.89      0.55      1515
    positive       0.67      0.44      0.53      1490

    accuracy                           0.50      4755
   macro avg       0.64      0.51      0.47      4755
weighted avg       0.65      0.50      0.46      4755



In [123]:
# As-Is with only Nouns
print("Airline Report : As-Is with NOUN POS")
report_airline_noun = run_vader_on_airline_tweets(lemmatize=False, pos={'NOUN'})
print(report_airline_noun)

Airline Report : As-Is with NOUN POS
              precision    recall  f1-score   support

    negative       0.73      0.14      0.23      1750
     neutral       0.36      0.82      0.50      1515
    positive       0.53      0.35      0.42      1490

    accuracy                           0.42      4755
   macro avg       0.54      0.44      0.39      4755
weighted avg       0.55      0.42      0.38      4755



In [124]:
# Lemmatized with only Nouns
print("Airline Report : Lemmatized with NOUN POS")
report_airline_lemmatized_noun = run_vader_on_airline_tweets(lemmatize=True, pos={'NOUN'})
print(report_airline_lemmatized_noun)

Airline Report : Lemmatized with NOUN POS
              precision    recall  f1-score   support

    negative       0.71      0.15      0.25      1750
     neutral       0.36      0.81      0.50      1515
    positive       0.52      0.34      0.41      1490

    accuracy                           0.42      4755
   macro avg       0.53      0.44      0.39      4755
weighted avg       0.54      0.42      0.38      4755



In [125]:
# As-Is with only Verbs
print("Airline Report : As-Is with VERB POS")
report_airline_verb = run_vader_on_airline_tweets(lemmatize=False, pos={'VERB'})
print(report_airline_verb)

Airline Report : As-Is with VERB POS
              precision    recall  f1-score   support

    negative       0.79      0.29      0.42      1750
     neutral       0.38      0.81      0.52      1515
    positive       0.57      0.35      0.43      1490

    accuracy                           0.47      4755
   macro avg       0.58      0.48      0.46      4755
weighted avg       0.59      0.47      0.46      4755



In [126]:
# Lemmatized with only Verbs
print("Airline Report : Lemmatized with VERB POS")
report_airline_lemmatized_verb = run_vader_on_airline_tweets(lemmatize=True, pos={'VERB'})
print(report_airline_lemmatized_verb)

Airline Report : Lemmatized with VERB POS
              precision    recall  f1-score   support

    negative       0.75      0.29      0.42      1750
     neutral       0.38      0.78      0.51      1515
    positive       0.57      0.36      0.44      1490

    accuracy                           0.47      4755
   macro avg       0.57      0.48      0.46      4755
weighted avg       0.58      0.47      0.46      4755



### Question 4b

When VADER is run As-Is, it has the best performance, with an accuracy of 0.63, meanwhile the Lemmatized result ends up being just slightly worse at 0.62, but it is not significant enough to make a noticeable difference. In this case, Lemmatization does not help. The reason could be that VADER does not recognise some Lemmatized words. Anywhere else, Lemmatization and As-Is does not have any difference at all.

Meanwhile, when using Parts-Of-Speech to measure sentiment analysis. We notice that adjectives have a slightly higher accuracy (0.50), with verbs coming second (0.47) and nouns having an accuracy of 0.42. This could mean that Adjectives are slightly more helpful at analysing sentiment, with verbs having a slightly less impactful (but still impactful) result.

## Part II: scikit-learn assignments
### [4 points] Question 5
Train the scikit-learn classifier (Naive Bayes) using the airline tweets.

+ Train the model on the airline tweets with 80% training and 20% test set and default settings (TF-IDF representation, min_df=2)
+ Train with different settings:
    + with respect to vectorizing: TF-IDF ('airline_tfidf') vs. Bag of words representation ('airline_count') 
    + with respect to the frequency threshold (min_df). Carry out experiments with increasing values for document frequency (min_df = 2; min_df = 5; min_df =10) 
* [1 point] a. Generate a classification_report for all experiments
* [3 points] b. Look at the results of the experiments with the different settings and try to explain why they differ: 
    + which category performs best, is this the case for any setting?
    + does the frequency threshold affect the scores? Why or why not according to you?

In [127]:
# Your code here


### [4 points] Question 6: Inspecting the best scoring features 

+ Train the scikit-learn classifier (Naive Bayes) model with the following settings (airline tweets 80% training and 20% test;  Bag of words representation ('airline_count'), min_df=2)
* [1 point] a. Generate the list of best scoring features per class (see function **important_features_per_class** below) [1 point]
* [3 points] b. Look at the lists and consider the following issues: 
    + [1 point] Which features did you expect for each separate class and why?
    + [1 point] Which features did you not expect and why ? 
    + [1 point] The list contains all kinds of words such as names of airlines, punctuation, numbers and content words (e.g., 'delay' and 'bad'). Which words would you remove or keep when trying to improve the model and why? 

In [128]:
def important_features_per_class(vectorizer,classifier,n=80):
    class_labels = classifier.classes_
    feature_names =vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]
    topn_class2 = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_class3 = sorted(zip(classifier.feature_count_[2], feature_names),reverse=True)[:n]
    print("Important words in negative documents")
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
    print("-----------------------------------------")
    print("Important words in neutral documents")
    for coef, feat in topn_class2:
        print(class_labels[1], coef, feat) 
    print("-----------------------------------------")
    print("Important words in positive documents")
    for coef, feat in topn_class3:
        print(class_labels[2], coef, feat) 

# example of how to call from notebook:
#important_features_per_class(airline_vec, clf)

### [Optional! (will not  be graded)] Question 7
Train the model on airline tweets and test it on your own set of tweets
+ Train the model with the following settings (airline tweets 80% training and 20% test;  Bag of words representation ('airline_count'), min_df=2)
+ Apply the model on your own set of tweets and generate the classification report
* [1 point] a. Carry out a quantitative analysis.
* [1 point] b. Carry out an error analysis on 10 correctly and 10 incorrectly classified tweets and discuss them
* [2 points] c. Compare the results (cf. classification report) with the results obtained by VADER on the same tweets and discuss the differences.

### [Optional! (will not be graded)] Question 8: trying to improve the model
* [2 points] a. Think of some ways to improve the scikit-learn Naive Bayes model by playing with the settings or applying linguistic preprocessing (e.g., by filtering on part-of-speech, or removing punctuation). Do not change the classifier but continue using the Naive Bayes classifier. Explain what the effects might be of these other settings 
+ [1 point] b. Apply the model with at least one new setting (train on the airline tweets using 80% training, 20% test) and generate the scores
* [1 point] c. Discuss whether the model achieved what you expected.

## End of this notebook